# detect someone with separation mood or emotional distress 

In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import threading
import warnings
warnings.filterwarnings('ignore', category=Warning)

In [7]:
s00 = pd.read_csv('../data/data_EEG/s00.csv',header=None)
s01 = pd.read_csv('../data/data_EEG/s01.csv',header=None)
s02 = pd.read_csv('../data/data_EEG/s02.csv',header=None)
s03 = pd.read_csv('../data/data_EEG/s03.csv',header=None)
s04 = pd.read_csv('../data/data_EEG/s04.csv',header=None)
s05 = pd.read_csv('../data/data_EEG/s05.csv',header=None)
s06 = pd.read_csv('../data/data_EEG/s06.csv',header=None)
s07 = pd.read_csv('../data/data_EEG/s07.csv',header=None)
s08 = pd.read_csv('../data/data_EEG/s08.csv',header=None)
s09 = pd.read_csv('../data/data_EEG/s09.csv',header=None)
s10 = pd.read_csv('../data/data_EEG/s10.csv',header=None)
s11 = pd.read_csv('../data/data_EEG/s11.csv',header=None)
s12 = pd.read_csv('../data/data_EEG/s12.csv',header=None)
s13 = pd.read_csv('../data/data_EEG/s13.csv',header=None)
s14 = pd.read_csv('../data/data_EEG/s14.csv',header=None)
s15 = pd.read_csv('../data/data_EEG/s15.csv',header=None)
s16 = pd.read_csv('../data/data_EEG/s16.csv',header=None)
s17 = pd.read_csv('../data/data_EEG/s17.csv',header=None)
s18 = pd.read_csv('../data/data_EEG/s18.csv',header=None)
s19 = pd.read_csv('../data/data_EEG/s19.csv',header=None)
s20 = pd.read_csv('../data/data_EEG/s20.csv',header=None)
s21 = pd.read_csv('../data/data_EEG/s21.csv',header=None)
s22 = pd.read_csv('../data/data_EEG/s22.csv',header=None)
s23 = pd.read_csv('../data/data_EEG/s23.csv',header=None)
s24 = pd.read_csv('../data/data_EEG/s24.csv',header=None)
s25 = pd.read_csv('../data/data_EEG/s25.csv',header=None)
s26 = pd.read_csv('../data/data_EEG/s26.csv',header=None)
s27 = pd.read_csv('../data/data_EEG/s27.csv',header=None)
s28 = pd.read_csv('../data/data_EEG/s28.csv',header=None)
s29 = pd.read_csv('../data/data_EEG/s29.csv',header=None)
s30 = pd.read_csv('../data/data_EEG/s30.csv',header=None)
s31 = pd.read_csv('../data/data_EEG/s31.csv',header=None)
s32 = pd.read_csv('../data/data_EEG/s32.csv',header=None)
s33 = pd.read_csv('../data/data_EEG/s33.csv',header=None)
s34 = pd.read_csv('../data/data_EEG/s34.csv',header=None)
s35 = pd.read_csv('../data/data_EEG/s35.csv',header=None)

In [8]:
s00.shape

(31000, 19)

# Leave-One-Out Cross-Validation (LOOCV) Explained

	Notice how rows 30-35 are excluded from the initial processing (like transpose and to_numpy) - these are reserved as the test set, not for validation!

	This method called Leave-One-Out Cross-Validation (LOOCV), is particularly valuable for EEG and other neuroimaging data because:

	Personalization Matters: Each participant has unique baseline brain activity, signal amplitudes, and response patterns

	Maximized Data Utility: With typical EEG studies having just 20-40 participants, LOOCV gives the most statistically reliable results

In [9]:
s00=s00.transpose().to_numpy()
s01=s01.transpose().to_numpy()
s02=s02.transpose().to_numpy()
s03=s03.transpose().to_numpy()
s04=s04.transpose().to_numpy()
s05=s05.transpose().to_numpy()
s06=s06.transpose().to_numpy()
s07=s07.transpose().to_numpy()
s08=s08.transpose().to_numpy()
s09=s09.transpose().to_numpy()
s10=s10.transpose().to_numpy()
s11=s11.transpose().to_numpy()
s12=s12.transpose().to_numpy()
s13=s13.transpose().to_numpy()
s14=s14.transpose().to_numpy()
s15=s15.transpose().to_numpy()
s16=s16.transpose().to_numpy()
s17=s17.transpose().to_numpy()
s18=s18.transpose().to_numpy()
s19=s19.transpose().to_numpy()
s20=s20.transpose().to_numpy()
s21=s21.transpose().to_numpy()
s22=s22.transpose().to_numpy()
s23=s23.transpose().to_numpy()
s24=s24.transpose().to_numpy()
s25=s25.transpose().to_numpy()
s26=s26.transpose().to_numpy()
s27=s27.transpose().to_numpy()
s28=s28.transpose().to_numpy()
s29=s29.transpose().to_numpy()

In [12]:
s00

array([[ 4.4760e+00,  1.2080e+00, -2.5460e+00, ..., -4.1361e-04,
        -4.1361e-04, -4.1361e-04],
       [-2.7403e+00, -3.3088e+00, -3.7083e+00, ..., -1.4025e-03,
        -1.4025e-03, -1.4025e-03],
       [-2.5017e+00, -4.4178e+00, -6.4109e+00, ...,  2.5287e-05,
         2.5287e-05,  2.5287e-05],
       ...,
       [ 3.3687e-02, -6.8218e-01, -1.2844e+00, ..., -4.0194e-04,
        -4.0194e-04, -4.0194e-04],
       [ 2.7164e+00,  4.8287e+00,  6.6513e+00, ..., -1.2302e-03,
        -1.2302e-03, -1.2302e-03],
       [ 8.4831e-01,  4.5261e-01, -1.0393e-01, ..., -9.8351e-04,
        -9.8351e-04, -9.8351e-04]])

# Original dataset with 36 samples

In [13]:

dataset= np.array(
    [[s00],[s01],[s02],[s03],[s04],[s05],[s06],[s07],
    [s08],[s09],[s10],[s11],[s12],[s13],[s14],[s15],[s16],[s17],[s18],[s19],
    [s20],[s21],[s22],[s23],[s24],[s25],[s26],[s27],
    [s28],[s29]]
)



In [15]:
dataset

array([[[[ 4.4760e+00,  1.2080e+00, -2.5460e+00, ..., -4.1361e-04,
          -4.1361e-04, -4.1361e-04],
         [-2.7403e+00, -3.3088e+00, -3.7083e+00, ..., -1.4025e-03,
          -1.4025e-03, -1.4025e-03],
         [-2.5017e+00, -4.4178e+00, -6.4109e+00, ...,  2.5287e-05,
           2.5287e-05,  2.5287e-05],
         ...,
         [ 3.3687e-02, -6.8218e-01, -1.2844e+00, ..., -4.0194e-04,
          -4.0194e-04, -4.0194e-04],
         [ 2.7164e+00,  4.8287e+00,  6.6513e+00, ..., -1.2302e-03,
          -1.2302e-03, -1.2302e-03],
         [ 8.4831e-01,  4.5261e-01, -1.0393e-01, ..., -9.8351e-04,
          -9.8351e-04, -9.8351e-04]]],


       [[[ 1.3990e+01,  1.2877e+01,  1.1885e+01, ...,  8.5942e-04,
           8.5942e-04,  8.5942e-04],
         [ 1.0550e+01,  1.0276e+01,  9.2391e+00, ...,  5.8796e-06,
           5.8796e-06,  5.8796e-06],
         [ 1.4146e+01,  1.2904e+01,  1.1402e+01, ..., -2.2769e-04,
          -2.2769e-04, -2.2769e-04],
         ...,
         [ 1.6089e+01,  1.5680e+

In [16]:
dataset.shape

(30, 1, 19, 31000)

# Reshape the dataset

In [20]:
dataset = dataset.reshape(30,1,760,775)